In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

그래프 한글깨짐 방지

In [2]:
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

In [3]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.6/774.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 9.7 MB/s eta 0:00:00


In [4]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.1 MB/s eta 0:00:00


In [5]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import FunnelTokenizerFast, FunnelModel
transformers.logging.set_verbosity_error()

In [6]:
import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [7]:
from argparse import ArgumentParser

parser = ArgumentParser(description="baseline")
#인수들과 기본값을 정의한다
parser.add_argument('--text_pretrained_model', default="kosroberta", type=str)
parser.add_argument('--truncation_side', default='right', type=str)
parser.add_argument('--pos_weight', default=None, type=int)
parser.add_argument('--optimizer', default="adamw", type=str)
parser.add_argument('--learning_rate', default=0.00003, type=float)
parser.add_argument('--scheduler', default="none", type=str)
parser.add_argument('--batch_size', default=16, type=int) #64 -> 16
parser.add_argument('--epochs', default=10, type=int)
parser.add_argument('--cv', default=5, type=int)
parser.add_argument('--seed', default=826, type=int)
parser.add_argument('--mixed_precision', default=16, type=int)
parser.add_argument('--device', default=1, type=int)
parser.add_argument('--num_workers', default=0, type=int)
args = parser.parse_args('')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

- argparse 모듈을 사용하여 명령줄 인수를 정의하고 구문을 분석
- 프로그램이 필요한 인자를 정의하면, argparse는 어떻게 파싱할지 파악

In [8]:
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
CV = args.cv
SEED = args.seed

In [9]:
def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(SEED)

set_seeds()

idx = f"{args.text_pretrained_model}_pw{args.pos_weight}"
idx

INFO:lightning_fabric.utilities.seed:Seed set to 826


'kosroberta_pwNone'

변수 값을 기반으로 text_pretrained_model변수 에 값을 할당하는 조건부 블록

In [10]:
if args.text_pretrained_model == "kosroberta":
    text_pretrained_model = "jhgan/ko-sroberta-multitask"
    latent_dim= 384 #* 2
if args.text_pretrained_model == "bert-kor-base":
    text_pretrained_model = "kykim/bert-kor-base"
    latent_dim= 384 #* 2
if args.text_pretrained_model == "funnel-kor-base":
    text_pretrained_model = "kykim/funnel-kor-base"
    latent_dim= 384 #* 2

# preprocessing

In [11]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/humanvsai/train.csv").drop(columns=["id"])
test_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/humanvsai/test.csv").drop(columns=["id"])

train_df.shape, test_df.shape

((50, 5), (1100, 4))

In [13]:
train_df.head(4)

,sentence1,sentence2,sentence3,sentence4,label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4


In [14]:
train_true = []
train_false = []
#리뷰를 사람이 썼는지 기계가 썼는지 0과 1로만 구분한다. 사람이 썼으면 1. 기계가 썼으면0이다
for i, v in enumerate(train_df["label"]):
    for j in range(1, 5):
        if v==j:
            train_true.append(train_df.iat[i, j-1])
        else:
            train_false.append(train_df.iat[i, j-1])

train_df_true = pd.DataFrame(train_true, columns=["text"])
train_df_false = pd.DataFrame(train_false, columns=["text"])
train_df_true["label"] = 1
train_df_false["label"] = 0

train_df = pd.concat([train_df_true, train_df_false]).reset_index(drop=True)
train_df.shape

(200, 2)

In [15]:
train_df.head(4)

,text,label
0,직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,1
1,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,1
2,일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,1
3,1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,1


In [16]:
#test데이터도 마찬가지로 인덱싱
test_df = test_df.reset_index()
test_df = pd.melt(test_df, id_vars='index', var_name="sentence", value_name="text")
test_df.shape

(4400, 3)

In [18]:
test_df.head(4)

,index,sentence,text
0,0,sentence1,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매..."
1,1,sentence1,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...
2,2,sentence1,"평일 저녁에 방문했을 때, 서비스가 좋지 않았습니다. 음료를 준비하는 듯 보여 기다..."
3,3,sentence1,"직원들의 친절도가 좋지 않았고, 손님이 오셨음에도 불구하고 무시하는 태도를 보여서 ..."


In [ ]:
# train의 문장과 정답 레이블을 분리하고 test의 문장도 따로 분리한다
X = train_df["text"].values
y = train_df["label"].values
X_test = test_df["text"].values

X.shape, y.shape, X_test.shape

((200,), (200,), (4400,))

# data_loader

In [ ]:
class TextDataset(Dataset):
    def __init__(self, sentence, label):
        self.sentence = sentence
        self.label = label
        #토크나이저 선택
        if args.text_pretrained_model == "funnel-kor-base":
            self.tokenizer = FunnelTokenizerFast.from_pretrained(text_pretrained_model)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model)
        if args.truncation_side == "left":
            self.tokenizer.truncation_side = 'left'

    def __len__(self):
        return len(self.sentence) #문장 수 반환 (데이터셋 길이)

    def __getitem__(self, idx):
        sentence = self.sentence[idx] #주어진 인덱스에서 문장을 가져옴

        encoded = self.tokenizer.encode_plus( #단어토큰화
            #Time is an illusion. Lunchtime double so!
            #=> "Time", "is", "an", "illustion", "Lunchtime", "double", "so"
            sentence,
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=True,
            return_attention_mask=True,
        )

        input_ids = encoded["input_ids"][0]
        token_type_ids = encoded["token_type_ids"][0]
        attention_masks = encoded["attention_mask"][0]

        if self.label is not None:
            label = self.label[idx]
            return [input_ids, token_type_ids, attention_masks], label
        else:
            return [input_ids, token_type_ids, attention_masks]

# model

In [ ]:
class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return F.silu(gate) * x

class TextModel(nn.Module):
    def __init__(self, latent_dim=latent_dim):
        super().__init__()
        if args.text_pretrained_model == "funnel-kor-base":
            self.txt_model = FunnelModel.from_pretrained(text_pretrained_model)
        else:
            self.txt_model = AutoModel.from_pretrained(text_pretrained_model)
        self.classifier = nn.Sequential(
            SwiGLU(),
            nn.Linear(latent_dim, 1, bias=False)
        )

    def forward(self, x):
        input_ids = x[0]
        token_type_ids = x[1]
        attention_mask = x[2]

        txt_side = self.txt_model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
        )

        txt_feature = txt_side.last_hidden_state[:, 0, :]

        output = self.classifier(txt_feature)

        return output

In [ ]:
class TextClassifier(pl.LightningModule):
    def __init__(self, backbone, args):
        super().__init__()
        self.backbone = backbone
        if args.pos_weight != None:
            self.pos_weight = torch.tensor(args.pos_weight)

    def forward(self, x):
        predictions = self.backbone(x)
        return predictions

    def step(self, batch):
        x, y = batch
        y_hat = self.backbone(x)
        if args.pos_weight != None:
            loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)(y_hat.squeeze(), y.float())
        else:
            loss = nn.BCEWithLogitsLoss()(y_hat.squeeze(), y.float())
        return loss, y, y_hat

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
        self.log("test_acc", acc)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        y_hat = self.backbone(batch)
        return y_hat

    def configure_optimizers(self):
        if args.optimizer == "sgd":
            optimizer = torch.optim.SGD(self.parameters(), lr=args.learning_rate, momentum=0.9)
        if args.optimizer == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=args.learning_rate)
        if args.optimizer == "adamw":
            optimizer = torch.optim.AdamW(self.parameters(), lr=args.learning_rate)

        if args.scheduler == "none":
            return optimizer
        if args.scheduler == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer=optimizer,
                step_size=2,
                gamma=0.9,
            )
            return [optimizer], [scheduler]
        if args.scheduler == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer=optimizer,
                T_max=EPOCHS,
                eta_min=1e-5,
            )
            return [optimizer], [scheduler]
        if args.scheduler == "onecyclelr":
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer=optimizer,
                max_lr=args.learning_rate,
                epochs=EPOCHS,
                steps_per_epoch=int(len(train_index) / BATCH_SIZE),
                pct_start=0.1,
            )
            return [optimizer], [scheduler]

# main

In [ ]:
## preprocessing.py

val_acc_list = []
preds_list = []

skf = StratifiedKFold(n_splits=CV, shuffle=True, random_state=SEED)

for i, (train_index, val_index) in enumerate(skf.split(X, y)):

    X_train = X[train_index]
    X_val = X[val_index]

    y_train = y[train_index]
    y_val = y[val_index]

## data_loaders.py

    train_ds = TextDataset(X_train, y_train)
    val_ds = TextDataset(X_val, y_val)
    test_ds = TextDataset(X_test, None)

    train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=args.num_workers)
    val_dataloader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)
    test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)

## train.py

    model = TextClassifier(TextModel(), args)

    callbacks = [
        pl.callbacks.ModelCheckpoint(
            dirpath="saved/", filename=f"{idx}_{i}",
            monitor="val_acc", mode="max"
        ),
    ]

    trainer = pl.Trainer(
        max_epochs=EPOCHS, accelerator="auto", callbacks=callbacks,
        precision=args.mixed_precision,
        #devices=[args.device]
    )

    trainer.fit(model, train_dataloader, val_dataloader)

    ckpt = torch.load(f"saved/{idx}_{i}.ckpt")
    model.load_state_dict(ckpt['state_dict'])

## test.py

    eval_dict = trainer.validate(model, dataloaders=val_dataloader)[0]
    val_acc_list.append(eval_dict["val_acc"])

    y_preds = trainer.predict(model, dataloaders=test_dataloader)
    preds_list.append(np.vstack(y_preds))

val_acc_mean = np.mean(val_acc_list)

print(f"val_acc_mean: {val_acc_mean}")

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name     | Type      | Params
---------------------------------------
0 | backbone | TextModel | 110 M 
---------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.474   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │     0.824999988079071     │
│         val_loss          │    0.4872126579284668     │
└───────────────────────────┴───────────────────────────┘

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

val_acc_mean: 0.9200000047683716


# Submission

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/humanvsai/sample_submission.csv')
y_pred = np.mean(preds_list, axis=0)
test_df["label"] = y_pred

submit_df = pd.pivot(test_df, columns="sentence", index="index", values="label")
submit_df.to_csv(f"test_{idx}.csv")

submit_df.head()

sentence,sentence1,sentence2,sentence3,sentence4
index,,,,
0,-3.275391,-1.001953,1.914062,1.775391
1,0.786621,-3.283203,-3.322266,-3.306641
2,-3.328125,-1.745117,-3.320312,-3.337891
3,-3.316406,-3.064453,-3.158203,-3.294922
4,-3.308594,-2.232422,1.933594,-3.312500


In [ ]:
submit_df = pd.DataFrame(
    submit_df.values,
    columns=['sentence1', 'sentence2', 'sentence3', 'sentence4'],
    dtype="float"
)

submit_df["label"] = submit_df.apply(lambda s: s.nlargest(2).index.tolist(), axis=1)
submit_df["label"] = submit_df["label"].apply(lambda x : ''.join(re.findall(r'\d+', str(x))))

submit_df.head()

,sentence1,sentence2,sentence3,sentence4,label
0,-3.275391,-1.001953,1.914062,1.775391,34
1,0.786621,-3.283203,-3.322266,-3.306641,12
2,-3.328125,-1.745117,-3.320312,-3.337891,23
3,-3.316406,-3.064453,-3.158203,-3.294922,23
4,-3.308594,-2.232422,1.933594,-3.312500,32


In [ ]:
submission["label"] = submit_df["label"]
submission.to_csv(f"submit_{idx}.csv", index=False)

submission["label"].value_counts()

21    100
32     99
13     99
42     97
34     96
12     94
24     89
43     88
14     88
23     84
41     84
31     82
Name: label, dtype: int64